In [ ]:
## summary

def preprocess_function(examples):
    inputs = examples["article"]
    targets = examples["highlights"]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=max_target_length, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    
    return model_inputs

tokenized_datasets = dataset.map(
        preprocess_function,
        batched=True,
        remove_columns=[]
    )

In [ ]:
## classify

class NewsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_texts, test_texts, train_labels, test_labels = train_test_split(data['short_description'], data['category'], test_size=0.2, shuffle=True)
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)
eval_labels_encoded = label_encoder.transform(eval_labels)

train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True)
train_labels_tensor = torch.tensor(train_labels_encoded)
train_dataset = NewsDataset(train_encodings, train_labels_tensor)

In [ ]:
## 

def preprocess(examples):
    return tokenizer(
        examples['text'],
        truncation=True,
        padding='max_length',
        max_length=256,
        return_tensors="pt"
    )

test_dataset = test_dataset.map(preprocess, batched=True)